[使い方説明はこちらから](https://spectrum-dugong-954.notion.site/XML-18aa1cc6aeb88010b6a6cfad43c690ca?pvs=73)

In [ ]:
pip install mecab-python3 unidic-lite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.8/588.8 kB 5.3 MB/s eta 0:00:00
  Created wheel for unidic-lite: filename=unidic_lite-1.0.8-py3-none-any.whl size=47658818 sha256=5dc6832ae259a18a25edd2a459a8edc6b3dd848f91d57f8f4bc4340554ccfafb
  Stored in directory: /root/.cache/pip/wheels/b7/fd/e9/ea4459b868e6d2902e8d80e82dbacb6203e05b3b3a58c64966
Successfully built unidic-lite


In [ ]:
import json
import collections

import xml.etree.ElementTree as ET
import re
from collections import Counter
import pandas as pd
import os
import MeCab
import csv
import glob






# カウント対象の品詞
EXCLUDED_POS_TAGS = [
    "名詞-固有名詞-一般",
    "名詞-普通名詞-サ変可能",
    "名詞-普通名詞-形状詞可能",
    "記号-文字",
    "名詞-固有名詞-人名-名",
    "名詞-固有名詞-人名-一般",
    "形状詞-一般",
    "名詞-固有名詞-地名-一般",
    "名詞-普通名詞-一般",
    "接尾辞-名詞的-一般"
]



def words_format(xml_file_path):
    """XMLファイルからテキストを抽出し、不要なタグ、空白等を削除する関数"""
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        text = ""

        # XMLタグ内のテキストを抽出
        for elem in root.iter():
            if elem.text:
                text += elem.text + " "

        # 不要な改行や空白を正規化
        text = re.sub(r'\s+', ' ', text).strip()
        return text



    except Exception as e:
        print(f"エラーが発生しました：{e}")
        return None






def save_dict_to_xlsx(data, filename="output.xlsx"):
  """
    辞書データをExcelファイルに保存する関数。

    Args:
        data (dict): 保存する辞書データ。
        filename (str, optional): 保存するファイル名. Defaults to "output.xlsx".
    """
  try:
    df = pd.DataFrame(list(data.items()), columns=['単語', '回数'])
    df.to_excel(filename, index=False, sheet_name='Sheet1')
    print(f"Successfully saved to {filename}")
  except Exception as e:
      print(f"Error saving to file: {e}")





def save_dict_to_txt(data, filename="output.txt"):
  """
  辞書データをCSV形式でtxtファイルに保存する関数。

  Args:
    data (dict): 保存する辞書データ。
    filename (str, optional): 保存するファイル名. Defaults to "output.txt".
  """
  try:
    with open(filename, 'w', encoding='utf-8', newline='') as f:
      writer = csv.writer(f)
      writer.writerow(['単語', '回数'])
      for word, count in data.items():
        writer.writerow([word, count])
    print(f"Successfully saved to {filename}")
  except Exception as e:
    print(f"Error saving to file: {e}")











def sort_dict_by_value_desc(data):
  """
  辞書を値（数値）の降順でソートした新しい辞書を返す関数。

  Args:
    data (dict): ソートする辞書データ。

  Returns:
    dict: 値でソートされた新しい辞書（OrderedDict）。
  """
  import collections

  sorted_items = sorted(data.items(), key=lambda item: item[1], reverse=True)
  return collections.OrderedDict(sorted_items)


def group_morphemes_by_pos(data):
    """
    形態素解析の結果を品詞ごとにまとめ、重複を排除する関数。

    Args:
        data (dict): 形態素解析の結果をまとめた辞書。

    Returns:
        dict: 品詞をキー、形態素のリストを値とした辞書。
    """
    grouped_morphemes = {}
    json_output_test_text = ""
    count_target_morphemes = [] # カウント対象とするリスト

    for key, value in data.items():
        morpheme = value[0]
        if len(value) > 4:
           pos = value[4]
        else:
            pos = ""
        json_output_test_text += pos + "：" + morpheme + "\n"
        if pos in EXCLUDED_POS_TAGS:  # 指定した品詞の場合のみカウント対象に追加
            count_target_morphemes.append(morpheme)
        if pos in grouped_morphemes:
            if morpheme not in grouped_morphemes[pos]:
              grouped_morphemes[pos].append(morpheme)
        else:
            grouped_morphemes[pos] = [morpheme]

    save_string_to_txt(json_output_test_text, filename="test_pos_morpheme.txt")
    return grouped_morphemes, count_target_morphemes









def save_to_json(data, filename="output.json"):
    """
    辞書データをJSONファイルとして保存する関数。

    Args:
        data (dict): 保存する辞書データ。
        filename (str, optional): 保存するファイル名. Defaults to "output.json".
    """
    try:
        with open(filename, 'w', encoding='utf-8') as f:
          json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"Successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving to file: {e}")








def save_string_to_txt(text, filename="output.txt"):
    """
    文字列変数をテキストファイルに保存する関数。

    Args:
        text (str): 保存する文字列。
        filename (str, optional): 保存するファイル名. Defaults to "output.txt".
    """
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(text)
        print(f"Successfully saved to {filename}")
    except Exception as e:
        print(f"Error saving to file: {e}")







def string_to_dict(text):
    """
    文字列を連想配列に変換する関数。

    Args:
        text (str): 変換する文字列。

    Returns:
        dict: 連想配列 (辞書)。
    """
    result_dict = {}
    lines = text.strip().split("\n")
    for index, line in enumerate(lines, start=1):  # start=1 で行番号を1から開始
       parts = line.split("\t")
       # 空文字を削除
       parts = [part for part in parts if part != ""]
       result_dict[index] = parts

    return result_dict












target_dir = '/'  # XMLファイルが格納されているフォルダのパスを設定（カレントディレクトリ）

xml_files = glob.glob(os.path.join(target_dir, '*.xml'))

if not xml_files:
    print("xmlファイルが存在しません。")
    exit(1)  # エラー終了
elif len(xml_files) > 1:
    print("xmlファイルが複数存在します。")
    exit(1)  # エラー終了
else:
    target_file_path = xml_files[0]
    print(f"Processing XML file: {target_file_path}") # 処理対象のファイルパスを表示








# MeCabによる形態素解析実行
wakati = MeCab.Tagger("-Owakati")
output_words = wakati.parse(target_words).split()

test_text = ""
for word in output_words:
  test_text += word + "\n"















target_words = words_format(target_file_path)

# MeCabによる形態素解析実行
normal_mecab = MeCab.Tagger()



output_words_dir = string_to_dict(normal_mecab.parse(target_words))




# 品詞毎にグループ化して.json出力
group_morphemes = group_morphemes_by_pos(output_words_dir)
group_morphemes, count_target_morphemes = group_morphemes_by_pos(output_words_dir)
save_to_json(group_morphemes, "grouped_word.json")










# 文字数カウントしてファイル×2出力

word_counter = {}

for word in count_target_morphemes:
  if word in word_counter:
    word_counter[word] += 1
  else:
    word_counter[word] = 1

word_counter = sort_dict_by_value_desc(word_counter)




word_counter = sort_dict_by_value_desc(word_counter)

save_dict_to_txt(word_counter, "output.txt")
save_dict_to_xlsx(word_counter, "output.xlsx")


Processing XML file: /347CO0000000318_20241118_506CO0000000342.xml
Successfully saved to test_mecab_owakati_process_after.txt
Successfully saved to test_mecab_process_before.txt
Successfully saved to test_not_split.txt
Successfully saved to test_pos_morpheme.txt
Successfully saved to test_pos_morpheme.txt
Successfully saved to grouped_word.json
Successfully saved to output.txt
Successfully saved to output.xlsx
